# Compare NLP Techniques: Build Model On doc2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Create doc2vec Vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets
tagged_docs_train = [gensim.models.doc2vec.TaggedDocument(v, [i])
                     for i, v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument(v, [i])
                    for i, v in enumerate(X_test['clean_text'])]

In [3]:
# What do these TaggedDocument objects look like?
tagged_docs_train[:10]

[TaggedDocument(words="['get', 'garden', 'ready', 'summer', 'free', 'selection', 'summer', 'bulbs', 'seeds', 'worth', 'å', '3350', 'scotsman', 'saturday', 'stop', 'go2', 'notxtcouk']", tags=[0]),
 TaggedDocument(words="['urgent', '1', 'week', 'free', 'membership', 'å', '100000', 'prize', 'jackpot', 'txt', 'word', 'claim', '81010', 'tc', 'wwwdbuknet', 'lccltd', 'pobox', '4403ldnw1a7rw18']", tags=[1]),
 TaggedDocument(words="['get', 'door', 'im']", tags=[2]),
 TaggedDocument(words="['win', 'newest', 'ûïharry', 'potter', 'order', 'phoenix', 'book', '5', 'reply', 'harry', 'answer', '5', 'questions', 'chance', 'first', 'among', 'readers']", tags=[3]),
 TaggedDocument(words="['idk', 'keep', 'saying', 'youre', 'since', 'moved', 'keep', 'butting', 'heads', 'freedom', 'vs', 'responsibility', 'im', 'tired', 'much', 'shit', 'deal', 'im', 'barely', 'keeping', 'together', 'gets', 'added']", tags=[4]),
 TaggedDocument(words="['hanging', 'brother', 'family']", tags=[5]),
 TaggedDocument(words="['simp

In [7]:
# Train a basic doc2vec model
d2v_model = gensim.models.Doc2Vec(tagged_docs_train,
                                  vector_size=100,
                                  window=5,
                                  min_count=2)

In [9]:
# Infer the vectors to be used in training and testing
train_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]
test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]

### Fit RandomForestClassifier On Top Of Document Vectors

In [10]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.951 / Recall: 0.387 / Accuracy: 0.915
